# 🎬 Avatar Renderer MCP - Google Colab Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruslanmv/avatar-renderer-mcp/blob/main/demo_colab.ipynb)

**Transform static images into dynamic AI-powered avatars with realistic expressions and voice synchronization**

This notebook demonstrates how to:
1. ✅ Set up Avatar Renderer MCP in Google Colab
2. ✅ Install and verify the MCP server
3. ✅ Generate "Hello World" talking avatars
4. ✅ Create professional demo videos
5. ✅ Use both quality modes (real-time & high-quality)

---

**⚡ Colab Pro Recommended**: For faster processing and GPU access

**🌐 Website:** [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)

**📦 Repository:** [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)

## ⚠️ Important Notes

### Runtime Requirements
- **Runtime Type**: Python 3
- **Hardware**: GPU recommended (Runtime → Change runtime type → GPU)
- **RAM**: High-RAM runtime recommended for high-quality mode
- **Disk Space**: ~10 GB for models and dependencies

### Expected Processing Times
- **Real-time mode**: 30-60 seconds (CPU) / 10-20 seconds (GPU)
- **High-quality mode**: 2-5 minutes (GPU required)

### Colab Limitations
- Session timeout after 12 hours (or 90 minutes idle)
- Models must be re-downloaded each session (or use Google Drive)
- GPU availability not guaranteed on free tier

## 1️⃣ Environment Setup & System Dependencies

First, let's check the Colab environment and install system dependencies.

In [ ]:
# Check Colab environment
import sys
import os
from pathlib import Path

print("🔍 Environment Information")
print("="*60)
print(f"Python Version: {sys.version}")
print(f"Working Directory: {os.getcwd()}")
print(f"Platform: {sys.platform}")

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠️  Not running in Colab (some features may not work)")

# Check GPU availability
import torch
cuda_available = torch.cuda.is_available()
print(f"\nGPU Available: {cuda_available}")
if cuda_available:
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("✅ GPU acceleration enabled")
else:
    print("⚠️  No GPU detected. Processing will be slower.")
    print("💡 Tip: Runtime → Change runtime type → GPU")

print("="*60)

In [ ]:
# Install system dependencies
print("📦 Installing system dependencies...\n")

!apt-get update -qq
!apt-get install -y -qq ffmpeg libsm6 libxext6 libxrender-dev libgomp1 > /dev/null 2>&1

# Verify FFmpeg installation
!ffmpeg -version | head -3

print("\n✅ System dependencies installed successfully!")

## 2️⃣ Clone Repository & Install Dependencies

Clone the Avatar Renderer MCP repository and install Python dependencies.

In [ ]:
# Clone the repository
import os
from pathlib import Path

REPO_DIR = Path("/content/avatar-renderer-mcp")

if REPO_DIR.exists():
    print("📂 Repository already cloned, pulling latest changes...")
    !cd {REPO_DIR} && git pull
else:
    print("📥 Cloning Avatar Renderer MCP repository...")
    !git clone https://github.com/ruslanmv/avatar-renderer-mcp.git {REPO_DIR}

# Change to repository directory
os.chdir(REPO_DIR)
print(f"\n✅ Working directory: {os.getcwd()}")

In [ ]:
# Install Python dependencies
print("📦 Installing Python dependencies...")
print("This may take 3-5 minutes...\n")

# Install the package in editable mode
!pip install -q -e .

# Install additional dependencies
!pip install -q ipython pillow matplotlib

print("\n✅ Python dependencies installed!")

## 3️⃣ Install External Dependencies (SadTalker, Wav2Lip, etc.)

Install the external Git repositories needed for avatar generation.

In [ ]:
# Create external dependencies directory
import sys
from pathlib import Path

EXT_DEPS_DIR = REPO_DIR / "external_deps"
EXT_DEPS_DIR.mkdir(exist_ok=True)

print("📦 Installing external dependencies...\n")

# Install SadTalker
sadtalker_dir = EXT_DEPS_DIR / "SadTalker"
if not sadtalker_dir.exists():
    print("📥 Cloning SadTalker...")
    !git clone --depth=1 https://github.com/OpenTalker/SadTalker.git {sadtalker_dir}
    !pip install -q -r {sadtalker_dir}/requirements.txt
else:
    print("✓ SadTalker already installed")

# Install First Order Motion Model (FOMM)
fomm_dir = EXT_DEPS_DIR / "first-order-model"
if not fomm_dir.exists():
    print("📥 Cloning First Order Motion Model...")
    !git clone --depth=1 https://github.com/AliaksandrSiarohin/first-order-model.git {fomm_dir}
else:
    print("✓ FOMM already installed")

# Install Wav2Lip
wav2lip_dir = EXT_DEPS_DIR / "Wav2Lip"
if not wav2lip_dir.exists():
    print("📥 Cloning Wav2Lip...")
    !git clone --depth=1 https://github.com/Rudrabha/Wav2Lip.git {wav2lip_dir}
    !pip install -q -r {wav2lip_dir}/requirements.txt
else:
    print("✓ Wav2Lip already installed")

# Add to Python path
if str(EXT_DEPS_DIR) not in sys.path:
    sys.path.insert(0, str(EXT_DEPS_DIR))
    print(f"\n✅ Added to Python path: {EXT_DEPS_DIR}")

print("\n✅ External dependencies installed!")

## 4️⃣ Download Model Checkpoints

Download the pre-trained model weights. This is the largest download (~3-5 GB).

**Note**: Models will be downloaded to Colab's temporary storage and need to be re-downloaded each session unless you mount Google Drive.

In [ ]:
# Option 1: Use temporary Colab storage (faster but lost after session)
MODELS_DIR = Path("/content/models")

# Option 2: Use Google Drive (persistent but slower)
# Uncomment the following lines to use Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# MODELS_DIR = Path("/content/drive/MyDrive/avatar-renderer-models")

MODELS_DIR.mkdir(exist_ok=True, parents=True)
os.environ['MODEL_ROOT'] = str(MODELS_DIR)

print(f"📁 Models directory: {MODELS_DIR}")
print(f"Storage: {'Google Drive (persistent)' if 'drive' in str(MODELS_DIR) else 'Colab storage (temporary)'}")

In [ ]:
# Download model checkpoints
import urllib.request
import gdown
from pathlib import Path

print("📥 Downloading model checkpoints...")
print("This will take 5-10 minutes depending on your connection...\n")

def download_file(url, output_path, description=""):
    """Download a file with progress bar"""
    output_path = Path(output_path)
    if output_path.exists():
        print(f"✓ {description} already exists, skipping...")
        return True
    
    output_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"📥 Downloading {description}...")
    
    try:
        if 'drive.google.com' in url:
            gdown.download(url, str(output_path), quiet=False, fuzzy=True)
        else:
            urllib.request.urlretrieve(url, output_path)
        print(f"✅ {description} downloaded")
        return True
    except Exception as e:
        print(f"❌ Failed to download {description}: {e}")
        return False

# Install gdown for Google Drive downloads
!pip install -q gdown

# Model URLs (these are examples - update with actual model URLs)
models_config = [
    {
        'name': 'SadTalker Checkpoint',
        'url': 'https://github.com/OpenTalker/SadTalker/releases/download/v0.0.1/checkpoints.zip',
        'path': MODELS_DIR / 'sadtalker' / 'checkpoints.zip',
        'extract': True
    },
    {
        'name': 'Wav2Lip GAN',
        'url': 'https://github.com/Rudrabha/Wav2Lip/releases/download/v1.0/wav2lip_gan.pth',
        'path': MODELS_DIR / 'wav2lip' / 'wav2lip_gan.pth',
        'extract': False
    },
    {
        'name': 'GFPGAN v1.3',
        'url': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
        'path': MODELS_DIR / 'gfpgan' / 'GFPGANv1.3.pth',
        'extract': False
    }
]

# Download models
success_count = 0
for model in models_config:
    if download_file(model['url'], model['path'], model['name']):
        success_count += 1
        
        # Extract if needed
        if model.get('extract') and model['path'].suffix == '.zip':
            print(f"📦 Extracting {model['name']}...")
            !unzip -q {model['path']} -d {model['path'].parent}

print(f"\n{'='*60}")
print(f"✅ Downloaded {success_count}/{len(models_config)} models")
if success_count < len(models_config):
    print("⚠️  Some models failed to download. You may experience limited functionality.")
print(f"{'='*60}")

## 5️⃣ Verify Installation & Test Imports

Let's verify that all components are installed correctly.

In [ ]:
# Verify installation
import sys
from pathlib import Path

print("🔍 Verifying installation...\n")
print("="*60)

# Check Python version
py_version = sys.version_info
print(f"Python Version: {py_version.major}.{py_version.minor}.{py_version.micro}")
if py_version >= (3, 11):
    print("✅ Python version compatible")
else:
    print("⚠️  Python 3.11+ recommended")

# Check imports
print("\n📦 Checking dependencies...")

imports_to_check = [
    ('torch', 'PyTorch'),
    ('torchvision', 'TorchVision'),
    ('fastapi', 'FastAPI'),
    ('librosa', 'Librosa'),
    ('soundfile', 'SoundFile'),
    ('gfpgan', 'GFPGAN'),
    ('diffusers', 'Diffusers'),
    ('transformers', 'Transformers'),
]

failed_imports = []
for module_name, display_name in imports_to_check:
    try:
        __import__(module_name)
        print(f"  ✅ {display_name}")
    except ImportError as e:
        print(f"  ❌ {display_name} - {e}")
        failed_imports.append(display_name)

# Check app module
print("\n📦 Checking app modules...")
try:
    from app.pipeline import render_pipeline
    print("  ✅ app.pipeline")
except ImportError as e:
    print(f"  ❌ app.pipeline - {e}")
    failed_imports.append('app.pipeline')

try:
    from app.settings import settings
    print("  ✅ app.settings")
except ImportError as e:
    print(f"  ❌ app.settings - {e}")

# Check directories
print("\n📁 Checking directories...")
dirs_to_check = [
    (REPO_DIR, "Repository"),
    (EXT_DEPS_DIR, "External dependencies"),
    (MODELS_DIR, "Models"),
]

for dir_path, name in dirs_to_check:
    if dir_path.exists():
        print(f"  ✅ {name}: {dir_path}")
    else:
        print(f"  ❌ {name} not found: {dir_path}")

# Summary
print("\n" + "="*60)
if not failed_imports:
    print("✅ All components verified successfully!")
    print("🎉 Ready to generate avatars!")
else:
    print(f"⚠️  {len(failed_imports)} components failed to load:")
    for name in failed_imports:
        print(f"  - {name}")
    print("\n💡 Try re-running the installation cells above")
print("="*60)

## 6️⃣ Setup & Test MCP Server

Now let's set up the MCP (Model Context Protocol) server and verify it's working correctly.

In [ ]:
# Setup environment variables for MCP server
import os

print("⚙️ Configuring MCP server...\n")

# Set environment variables
env_vars = {
    'MODEL_ROOT': str(MODELS_DIR),
    'PYTHONPATH': f"{REPO_DIR}:{EXT_DEPS_DIR}:{os.environ.get('PYTHONPATH', '')}",
    'LOG_LEVEL': 'INFO',
    'TMP_DIR': '/tmp',
    'CUDA_VISIBLE_DEVICES': '0' if torch.cuda.is_available() else '',
}

for key, value in env_vars.items():
    os.environ[key] = value
    print(f"✓ {key}={value[:50]}{'...' if len(value) > 50 else ''}")

print("\n✅ MCP server configured!")

In [ ]:
# Test MCP server components
print("🧪 Testing MCP server components...\n")

try:
    # Test pipeline import
    from app.pipeline import render_pipeline
    print("✅ Pipeline module loaded")
    
    # Test settings
    from app.settings import settings
    print("✅ Settings module loaded")
    print(f"   Model root: {settings.model_root if hasattr(settings, 'model_root') else 'Not set'}")
    
    # Test API (optional)
    try:
        from app.api import app
        print("✅ API module loaded")
    except Exception as e:
        print(f"⚠️  API module: {e}")
    
    # Test MCP server
    try:
        from app.mcp_server import render_avatar_tool
        print("✅ MCP server module loaded")
    except Exception as e:
        print(f"⚠️  MCP server: {e}")
    
    print("\n✅ MCP server components verified!")
    print("🎬 Ready to generate avatars!")
    
except Exception as e:
    print(f"❌ Error loading MCP components: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Make sure all installation cells completed successfully")
    print("2. Check that models are downloaded")
    print("3. Verify Python path includes external dependencies")
    import traceback
    traceback.print_exc()

## 7️⃣ Generate Your First Avatar - "Hello World"

Let's create a simple "Hello World" talking avatar using the test assets!

In [ ]:
# Setup output directory
OUTPUT_DIR = Path("/content/avatar_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"📁 Output directory: {OUTPUT_DIR}")

# Check for test assets
test_image = REPO_DIR / "tests/assets/alice.png"
test_audio = REPO_DIR / "tests/assets/hello.wav"

print("\n🎨 Test assets:")
if test_image.exists():
    print(f"  ✅ Image: {test_image}")
    # Display the test image
    from IPython.display import Image, display
    display(Image(filename=str(test_image), width=300))
else:
    print(f"  ❌ Image not found: {test_image}")

if test_audio.exists():
    print(f"  ✅ Audio: {test_audio}")
    # Play the test audio
    from IPython.display import Audio, display
    display(Audio(filename=str(test_audio)))
else:
    print(f"  ❌ Audio not found: {test_audio}")

In [ ]:
# Generate Hello World avatar
import time
from IPython.display import Video, display

output_video = OUTPUT_DIR / "hello_world.mp4"

print("🎬 Generating 'Hello World' Avatar")
print("="*60)
print(f"Input Image: {test_image.name}")
print(f"Input Audio: {test_audio.name}")
print(f"Output: {output_video.name}")
print(f"Quality Mode: auto")
print("="*60)
print("\n⏳ Processing... This may take 30-60 seconds...\n")

start_time = time.time()

try:
    # Import and run the pipeline
    from app.pipeline import render_pipeline
    
    result = render_pipeline(
        face_image=str(test_image),
        audio=str(test_audio),
        out_path=str(output_video),
        quality_mode="auto"  # Auto-select best mode
    )
    
    processing_time = time.time() - start_time
    
    print(f"\n{'='*60}")
    print("✅ SUCCESS! Avatar generated!")
    print(f"{'='*60}")
    print(f"⏱️  Processing time: {processing_time:.2f} seconds")
    print(f"📁 Output file: {result}")
    print(f"📊 File size: {Path(result).stat().st_size / (1024*1024):.2f} MB")
    print(f"{'='*60}\n")
    
    # Display the video
    print("🎥 Your AI Avatar:")
    display(Video(result, width=640, embed=True))
    
except Exception as e:
    processing_time = time.time() - start_time
    print(f"\n{'='*60}")
    print("❌ ERROR: Avatar generation failed")
    print(f"{'='*60}")
    print(f"Error: {str(e)}")
    print(f"Time elapsed: {processing_time:.2f} seconds")
    print(f"\n🔧 Troubleshooting:")
    print("1. Check that models are downloaded correctly")
    print("2. Verify test assets exist")
    print("3. Check GPU availability for high-quality mode")
    print("4. Try quality_mode='real_time' for CPU-only systems")
    print(f"{'='*60}\n")
    
    # Print full traceback for debugging
    import traceback
    print("\n📋 Full error trace:")
    traceback.print_exc()

## 8️⃣ Test Different Quality Modes

Let's compare the two quality modes:

| Feature | Real-Time | High-Quality |
|---------|-----------|-------------|
| **Speed** | <3s (local) / 30-60s (Colab) | 2-5 minutes |
| **GPU Required** | No | Yes |
| **Pipeline** | SadTalker + Wav2Lip | FOMM + Diff2Lip + GFPGAN |

In [ ]:
# Test Real-Time Mode (CPU-friendly)
import time
from IPython.display import Video, display

output_realtime = OUTPUT_DIR / "realtime_demo.mp4"

print("⚡ Testing REAL-TIME Mode")
print("="*60)
print("Optimized for speed, works on CPU")
print("="*60)

start_time = time.time()

try:
    result = render_pipeline(
        face_image=str(test_image),
        audio=str(test_audio),
        out_path=str(output_realtime),
        quality_mode="real_time"
    )
    
    processing_time = time.time() - start_time
    print(f"\n✅ Real-time mode completed in {processing_time:.2f}s")
    display(Video(result, width=640, embed=True))
    
except Exception as e:
    print(f"❌ Real-time mode failed: {e}")
    print("This mode requires SadTalker and Wav2Lip models")

In [ ]:
# Test High-Quality Mode (GPU required)
import time
from IPython.display import Video, display

if not torch.cuda.is_available():
    print("⚠️  High-quality mode requires GPU")
    print("💡 Enable GPU: Runtime → Change runtime type → GPU")
    print("Skipping high-quality test...")
else:
    output_hq = OUTPUT_DIR / "highquality_demo.mp4"
    
    print("🎨 Testing HIGH-QUALITY Mode")
    print("="*60)
    print("Best quality with GFPGAN enhancement")
    print("="*60)
    
    start_time = time.time()
    
    try:
        result = render_pipeline(
            face_image=str(test_image),
            audio=str(test_audio),
            out_path=str(output_hq),
            quality_mode="high_quality"
        )
        
        processing_time = time.time() - start_time
        print(f"\n✅ High-quality mode completed in {processing_time:.2f}s")
        display(Video(result, width=640, embed=True))
        
    except Exception as e:
        print(f"❌ High-quality mode failed: {e}")
        print("This mode requires FOMM, Diff2Lip, and GFPGAN models")

## 9️⃣ Create Custom Avatars

Upload your own images and audio files to create custom avatars!

In [ ]:
# Upload custom files
from google.colab import files
import shutil

UPLOAD_DIR = Path("/content/uploads")
UPLOAD_DIR.mkdir(exist_ok=True)

print("📤 Upload your files:")
print("1. Click the 'Choose Files' button below")
print("2. Select an image (PNG/JPG) and audio file (WAV/MP3)")
print("="*60)

uploaded = files.upload()

# Move uploaded files
uploaded_files = []
for filename in uploaded.keys():
    src = Path(filename)
    dst = UPLOAD_DIR / filename
    shutil.move(str(src), str(dst))
    uploaded_files.append(dst)
    print(f"✅ Uploaded: {filename}")

print(f"\n📁 Files saved to: {UPLOAD_DIR}")

In [ ]:
# Generate avatar from uploaded files
from IPython.display import Video, display
import time

# Find uploaded image and audio
image_files = list(UPLOAD_DIR.glob("*.png")) + list(UPLOAD_DIR.glob("*.jpg"))
audio_files = list(UPLOAD_DIR.glob("*.wav")) + list(UPLOAD_DIR.glob("*.mp3"))

if not image_files or not audio_files:
    print("⚠️  Please upload both an image and an audio file")
else:
    custom_image = image_files[0]
    custom_audio = audio_files[0]
    custom_output = OUTPUT_DIR / "custom_avatar.mp4"
    
    print("🎬 Generating custom avatar...")
    print(f"  Image: {custom_image.name}")
    print(f"  Audio: {custom_audio.name}")
    
    start_time = time.time()
    
    try:
        result = render_pipeline(
            face_image=str(custom_image),
            audio=str(custom_audio),
            out_path=str(custom_output),
            quality_mode="auto"
        )
        
        processing_time = time.time() - start_time
        print(f"\n✅ Custom avatar generated in {processing_time:.2f}s")
        display(Video(result, width=640, embed=True))
        
    except Exception as e:
        print(f"❌ Failed: {e}")

## 🔟 Download Your Videos

Download the generated videos to your local machine.

In [ ]:
# Download generated videos
from google.colab import files

print("📥 Available videos for download:")
print("="*60)

video_files = list(OUTPUT_DIR.glob("*.mp4"))

if not video_files:
    print("No videos found. Generate some avatars first!")
else:
    for video in video_files:
        size_mb = video.stat().st_size / (1024 * 1024)
        print(f"  📹 {video.name} ({size_mb:.2f} MB)")
    
    print("\n💾 Click below to download each video:")
    for video in video_files:
        print(f"\nDownloading: {video.name}")
        files.download(str(video))

## 💡 Tips & Best Practices

### 🖼️ Image Requirements
- **Format**: PNG or JPG
- **Face**: Clear, front-facing portrait
- **Resolution**: Minimum 512x512, recommended 1024x1024
- **Lighting**: Well-lit, avoid harsh shadows

### 🎤 Audio Requirements
- **Format**: WAV or MP3
- **Sample Rate**: 16kHz recommended
- **Quality**: Clear speech, minimal background noise
- **Length**: Keep it concise (10-30 seconds optimal)

### ⚡ Performance Tips
1. **Use GPU**: Enable GPU runtime for 10x faster processing
2. **Real-time mode**: Use for faster processing on CPU
3. **High-quality mode**: Use GPU runtime for best results
4. **Google Drive**: Mount Drive to persist models across sessions
5. **Colab Pro**: Consider upgrading for longer sessions and better GPUs

### 🔧 Troubleshooting

| Issue | Solution |
|-------|----------|
| Out of memory | Use real_time mode or restart runtime |
| Slow processing | Enable GPU runtime |
| Models not found | Re-run model download cells |
| Import errors | Re-run installation cells |
| Session timeout | Mount Google Drive to save models |

### 📚 Additional Resources
- **Documentation**: [README.md](https://github.com/ruslanmv/avatar-renderer-mcp)
- **Website**: [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)
- **Issues**: [GitHub Issues](https://github.com/ruslanmv/avatar-renderer-mcp/issues)
- **Author**: [Ruslan Magana Vsevolodovna](https://ruslanmv.com)

## 🔌 MCP Server API Reference

### Core Function

```python
render_pipeline(
    face_image: str,              # Path to avatar image
    audio: str,                   # Path to audio file
    out_path: str,                # Output video path
    reference_video: str = None,  # Optional driving video
    quality_mode: str = "auto"    # "real_time", "high_quality", "auto"
) -> str  # Returns path to generated video
```

### Quality Modes

#### `real_time`
- Fast processing, CPU-friendly
- Pipeline: SadTalker + Wav2Lip
- Best for: Live streaming, chatbots

#### `high_quality`
- Maximum quality with GFPGAN
- Pipeline: FOMM + Diff2Lip + GFPGAN
- Best for: Marketing videos, YouTube

#### `auto` (default)
- Automatically selects best mode
- Based on GPU and model availability

## 🎉 Congratulations!

You've successfully set up Avatar Renderer MCP in Google Colab and generated your first AI-powered talking avatar!

### Next Steps

1. **Create more avatars** with your own images and audio
2. **Experiment with quality modes** to find the best balance
3. **Deploy to production** using the Docker/Kubernetes guides
4. **Integrate with your app** using the REST API or MCP protocol

### Support & Contact

- 🌐 **Website**: [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)
- 📦 **Repository**: [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)
- 📧 **Email**: contact@ruslanmv.com
- 👤 **Author**: [Ruslan Magana Vsevolodovna](https://ruslanmv.com)

---

*Transform static images into dynamic, AI-powered avatars with realistic expressions and voice synchronization* 🎬✨